In [1]:
from xml.dom import minidom
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import random

In [2]:
xml_dir = r"E:\DataSets\carPlatesDataset\letterBoxes"
img_dir = r"E:\DataSets\carPlatesDataset\raw_plates"
columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
xml_names = os.listdir(xml_dir)
img_names = os.listdir(img_dir)

In [3]:
xml_names

['IMG_20220330_121505_0.xml',
 'IMG_20220330_121505_1.xml',
 'IMG_20220330_121509_0.xml',
 'IMG_20220330_121509_1.xml',
 'IMG_20220330_121509_2.xml',
 'IMG_20220330_121518_0.xml',
 'IMG_20220330_121518_1.xml',
 'IMG_20220330_121704_0.xml',
 'IMG_20220330_121706_0.xml',
 'IMG_20220330_121706_2.xml',
 'IMG_20220330_121708_0.xml',
 'IMG_20220330_121708_2.xml',
 'IMG_20220330_121711_0.xml',
 'IMG_20220330_121711_1.xml',
 'IMG_20220330_122048_2.xml',
 'IMG_20220330_122115_0.xml',
 'IMG_20220330_122115_1.xml',
 'IMG_20220330_122115_2.xml',
 'IMG_20220330_122115_3.xml',
 'IMG_20220330_122117_0.xml',
 'IMG_20220330_122117_2.xml',
 'IMG_20220330_122238_0.xml',
 'IMG_20220330_122241_0.xml',
 'IMG_20220330_122257_0.xml',
 'IMG_20220330_122308_0.xml',
 'IMG_20220330_122337_0.xml',
 'IMG_20220330_122337_1.xml',
 'IMG_20220330_122344_1.xml',
 'IMG_20220330_122344_4.xml',
 'IMG_20220330_122424_0.xml',
 'IMG_20220330_122424_1.xml',
 'IMG_20220330_122508_0.xml',
 'IMG_20220330_122648_0.xml',
 'IMG_2022

In [4]:
len(xml_names)

419

In [5]:
class xml:
    def __init__(self, directory):
        self.doc = minidom.parse(directory)
        self.file_name = directory.split("\\")[-1].split(".")[0]+'.jpg'
        self.img_path = os.path.join(img_dir, self.file_name)
        sizes = self.doc.getElementsByTagName('size')
        self.width = self.get_size()[0]
        self.height = self.get_size()[1]
        self.bndbox = self.get_bndbox()
        self.df_elements = self.get_df_elements()
        
    def get_size(self):
        sizes = self.doc.getElementsByTagName('size')
        width = int(str(sizes[0].getElementsByTagName('width')[0].firstChild).split("'")[1])
        height = int(str(sizes[0].getElementsByTagName('height')[0].firstChild).split("'")[1])
        return [width, height]
    
    def get_bndbox(self):
        objs = self.doc.getElementsByTagName('object')
        arr = []
        for obj in objs:
            bndbox = obj.getElementsByTagName('bndbox')[0]
            xmin = int(str(bndbox.getElementsByTagName('xmin')[0].firstChild).split("'")[1])
            ymin = int(str(bndbox.getElementsByTagName('ymin')[0].firstChild).split("'")[1])
            xmax = int(str(bndbox.getElementsByTagName('xmax')[0].firstChild).split("'")[1])
            ymax = int(str(bndbox.getElementsByTagName('ymax')[0].firstChild).split("'")[1])
            cls = str(obj.getElementsByTagName('name')[0].firstChild).split("'")[1]
            arr.append([xmin, ymin, xmax, ymax, cls])
        return arr
    
    def get_df_elements(self):
        elements = []
        for bndbox in self.bndbox:
            elements.append([self.img_path,
                             self.width,
                             self.height,
                             bndbox[4],
                             bndbox[0], bndbox[1], bndbox[2], bndbox[3]])
        return elements

In [6]:
xml_names[0]

'IMG_20220330_121505_0.xml'

In [7]:
columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']

In [8]:
f = xml(os.path.join(xml_dir, xml_names[0]))
f.file_name

'IMG_20220330_121505_0.jpg'

In [9]:
train_df = pd.DataFrame(columns = columns)
test_df = pd.DataFrame(columns = columns)
random.shuffle(xml_names)
split = int(1*len(xml_names))

for xml_name in xml_names[:split]:
    file = xml(os.path.join(xml_dir, xml_name))
    if len(file.df_elements) == 0:
        print("Fuck")
    for element in file.df_elements:
        train_df.loc[len(train_df)] = element
        
for xml_name in xml_names[split:]:
    file = xml(os.path.join(xml_dir, xml_name))
    for element in file.df_elements:
        test_df.loc[len(test_df)] = element

In [12]:
data_dir = r"C:\Users\PC\Documents\projects\carPlates\data"
data_ver = "letters420"
train_labels_name = "train_labels.csv"
test_labels_name = "test_labels_mod.csv"

In [13]:
train_df.to_csv(os.path.join(data_dir, data_ver, train_labels_name))
train_df.head()


,filename,width,height,class,xmin,ymin,xmax,ymax
0,E:\DataSets\carPlatesDataset\raw_plates\IMG_20...,491,430,letter,395,211,474,325
1,E:\DataSets\carPlatesDataset\raw_plates\IMG_20...,491,430,letter,330,229,392,305
2,E:\DataSets\carPlatesDataset\raw_plates\IMG_20...,491,430,letter,252,218,314,311
3,E:\DataSets\carPlatesDataset\raw_plates\IMG_20...,491,430,letter,146,192,198,308
4,E:\DataSets\carPlatesDataset\raw_plates\IMG_20...,491,430,letter,94,192,131,307


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2560 entries, 0 to 2559
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  2560 non-null   object
 1   width     2560 non-null   object
 2   height    2560 non-null   object
 3   class     2560 non-null   object
 4   xmin      2560 non-null   object
 5   ymin      2560 non-null   object
 6   xmax      2560 non-null   object
 7   ymax      2560 non-null   object
dtypes: object(8)
memory usage: 180.0+ KB


In [15]:
test_df.to_csv(os.path.join(data_dir, data_ver, test_labels_name))
test_df.head()


,filename,width,height,class,xmin,ymin,xmax,ymax


In [16]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  0 non-null      object
 1   width     0 non-null      object
 2   height    0 non-null      object
 3   class     0 non-null      object
 4   xmin      0 non-null      object
 5   ymin      0 non-null      object
 6   xmax      0 non-null      object
 7   ymax      0 non-null      object
dtypes: object(8)
memory usage: 0.0+ bytes
